A good tool makes simple things easy and complex things possible.

If you don't know which solution is better, start with the one that's simpler.

In [1]:
import os
from yaml import load, CLoader as Loader

data = load(open('env/local-env.yml'), Loader=Loader)
os.environ.update({key: str(val) for key, val in data.items()})

from hemlock import push_app_context

push_app_context()

<Flask 'hemlock.app'>

In [12]:
from hemlock import Compile, Input, Label, Page, Submit

from random import randint

POT = 20

def gen_check_page(accept):
    return Compile.clear_response(Compile.random_proposal(
        Page(
            Label(),
            Input(
                '<p>How much money does the proposer receive?</p>',
                prepend='$',
                append='.00'
            ),
            Input(
                '<p>How much money does the responder receive?</p>',
                prepend='$',
                append='.00'
            )
        ),
        accept=accept
    ))

@Compile.register
def random_proposal(page, accept):
    # randomly generate a proposed split and response
    n = randint(1, POT-1)
    proposal = n, POT-n
    response = randint(0, POT-n-1) if accept else randint(POT-n, POT)
    # compute the payoff
    payoff = proposal if response<=proposal[1] else (0, 0)
    # describe the proposal and response in the label
    page.questions[0].label = '''
    <p>Imagine the proposer proposes the following split:</p>
    <ul>
       <li>Proposer: ${}</li>
       <li>Responder: ${}</li>
    </ul>
    <p>The responder says, "I will accept any proposal which gives 
    me at least ${}."</p>
    '''.format(*proposal, response)
    # add submit functions to verify that the response was correct
    page.questions[1].submit_functions.clear()
    Submit.match(page.questions[1], str(payoff[0]))
    page.questions[2].submit_functions.clear()
    Submit.match(page.questions[2], str(payoff[1]))

page = gen_check_page(accept=True)
path = page._compile()#.preview()

(13, 7)


In [9]:
os.remove(path)